<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Web Scraping OpenTable With Selenium: Guided Lab

_Authors: Joseph Nelson (DC)_

---

> *Note: This is intended to be an instructor-guided lab.*


In today's code-along lab, we'll build a scraper using urllib and Beautiful Soup. We'll also remedy some of the pitfalls of automated scraping by using a "headless" browser called Selenium.

You'll be scraping OpenTable's Washington, D.C. listings. We're interested in knowing the restaurants' **name, location, price, and how many people booked it that day.**

OpenTable provides all of this information on this page: http://www.opentable.com/washington-dc-restaurant-listings.

### 1) Inspect the elements of this page to confirm we can find all of the information we're interested in.

### 2) Use `urllib` and `BeautifulSoup` to read the contents of the HTML.

In [5]:
from bs4 import BeautifulSoup
import urllib
import requests

In [6]:
# Set the URL we want to visit.
url = "http://www.opentable.com/washington-dc-restaurant-listings"

response = requests.get(url)

# You can use status codes to understand how the target server responds to your request.
# Ex., 200 = OK, 400 = Bad Request, 403 = Forbidden, 404 = Not Found.
print 'Status Code: ',response.status_code

# Pull the HTML string out of requests and convert it to a Python string.
html = response.text

Status Code:  200


### 3) Print out a fraction of the HTML. What's in it?

In [8]:
# The first 500 characters of the content.
print "\nFirst part of HTML document fetched as string:\n"
print html


First part of HTML document fetched as string:

           <!DOCTYPE html><html lang="en"><head><meta charset="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE"/> <title>Restaurant Reservation Availability</title>    <meta  name="robots" content="noindex" > </meta><link  rel="canonical" href="https://www.opentable.com/washington-dc-restaurant-listings" > </link>      <link rel="shortcut icon" href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon.ico" type="image/x-icon"/><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-16.png" sizes="16x16"/><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-32.png" sizes="32x32"/><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-48.png" sizes="48x48"/><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-64.png" sizes="64x64"/><link rel="icon" href="

### 4) Use Beautiful Soup to convert the raw HTML into a soup object.

In [9]:
soup = BeautifulSoup(html, 'lxml')

### 5) Extract the name of each restaurant.

First, let's find each restaurant name listed on the page we've loaded. How do we find each restaurant's location on the page? 

> *Hint: We need to know where the restaurant element is housed in the **HTML**.*

**5.A) See if you can find the restaurant name. Keep in mind that there are many restaurants loaded on the page.**

In [23]:
spans = soup.findAll("span", {"class": "rest-row-name-text"})

**5.B) Create a list of _only_ the restaurant names (no tags).**


In [33]:
rest_name = [span.get_text() for span in spans]
rest_name[:5]

[u'Grayce Moen', u'Knolls', u'Okeys', u'Omnis Forge', u'Vel']

### 6) Repeat this process for location.

For example, barmini by Jose Andres is located in "Penn Quarter," as listed in our search results.

In [34]:
spans = soup.findAll("span", {"class": "rest-row-meta--location rest-row-meta-text"})

In [62]:
rest_loc = [span.text for span in spans]
rest_loc[:5]
(pd.DataFrame(rest_loc) == "Penn Quarter").sum()

0    0
dtype: int64

### 7) Get the price for each restaurant.

The price is the number of dollar signs on a scale of one to four for each restaurant. We'll follow the same process we used for restaurant name and location.

In [38]:
divy = soup.findAll("div", {"class": "rest-row-pricing"})

In [86]:
rest_pr = [div.text for div in divy]
rest_pr

for rest in rest_pr:
    print rest.replace('       $        ','').replace('         $    $      ','').replace('    ','')

   $$$
   $$$$   
   $$$
   $$$$   
   $$$
   $$$$   
   $$$$   
   $$$$   
   $$$
   $$$
   $$
   $$$$   
   $$
   $$$
   $$
   $$$
   $$
   $$$$   
   $$$
   $$
   $$$
   $$$
   $$$$   
   $$$$   
   $$
   $$
   $$$
   $$$$   
   $$$
   $$
   $$$
   $$$
   $$$
   $$$
   $$$
   $$$
   $$$$   
   $$$$   
   $$
   $$$$   
   $$
   $$
   $$
   $$
   $$$$   
   $$$
   $$
   $$$
   $$$$   
   $$$
   $$
   $$$$   
   $$
   $$
   $$$$   
   $$$
   $$$
   $$$$   
   $$$$   
   $$
   $$
   $$$$   
   $$$
   $$$
   $$
   $$$
   $$
   $$
   $$$$   
   $$$$   
   $$$
   $$$$   
   $$$
   $$$
   $$$$   
   $$$
   $$
   $$
   $$$
   $$
   $$$
   $$
   $$
   $$
   $$$$   
   $$
   $$
   $$
   $$
   $$$
   $$$
   $$$
   $$$
   $$$$   
   $$$
   $$$$   
   $$$
   $$$$   
   $$$
   $$$


In [67]:
rest_pr2 = []
for pr in rest_pr:
    rest_pr2.append(pr.replace(" ","x").replace('xxxxxxx$xxxxxxxx', '').replace('xxxxxxxxx$xxxx$xxxxxx', '').replace('x', ''))

In [68]:
rest_pr2[:5]

[u'$$$', u'$$$$', u'$$$', u'$$$$', u'$$$']

**7.B) Convert the dollar sign strings to a count of the number of dollar signs.**

Can you figure out a way to print out the number of dollar signs per restaurant listed?

In [71]:
rest_pr_ct = []
for pr in rest_pr2:
    rest_pr_ct.append(pr.count("$"))

In [74]:
rest_pr_ct[:5]

[3, 4, 3, 4, 3]

### 8) Can you find the number of times a restaurant was booked?

In the next cell, print out a sample of objects that contain the number of times a restaurant was booked.

> *Note: If you can't, why do you think this happens?*

In [79]:
spans = soup.findAll("div", {"class": "booking"})

In [80]:
spans

[]

In [81]:
soup

<!DOCTYPE html>\n<html lang="en"><head><meta charset="unicode-escape"/><meta content="IE=9; IE=8; IE=7; IE=EDGE" http-equiv="X-UA-Compatible"/> <title>Restaurant Reservation Availability</title> <meta content="noindex" name="robots"/> <link href="https://www.opentable.com/washington-dc-restaurant-listings" rel="canonical"/> <link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon.ico" rel="shortcut icon" type="image/x-icon"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-16.png" rel="icon" sizes="16x16"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-32.png" rel="icon" sizes="32x32"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-48.png" rel="icon" sizes="48x48"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-64.png" rel="icon" sizes="64x64"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-128.png" rel="icon

## Enter: Selenium

---

Selenium is a headless browser. That means it enables us to mock human-browsing behavior — it even waits for JavaScript elements to load.

If you don't already have Selenium installed, you can do so via pip. Simply run `pip install selenium`.

In [82]:
!pip install selenium

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [83]:
# Import:
from selenium import webdriver

To run, Selenium requires us to determine a default browser. We're going to opt for Firefox, but Chromium is also a very common choice.

http://selenium-python.readthedocs.io/faq.html

### 9) What's going to happen when we run the next cell?

The ChromeDriver has been provided in the "chromedriver" folder, so don't worry about downloading another one.

In [103]:
# NOTE: YOU MAY NEED TO  RUN THIS

import os
from selenium import webdriver

#CHANGE THIS WITH THE RIGHT PATH!
chromedriver = "Desktop/classes/week-06/labs/python-webscraping_opentable-lab-master/chromedriver/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [104]:
# Create a driver called "driver."
driver = webdriver.Chrome(executable_path="C:/Users/Samson/Downloads/chromedriver_win32/chromedriver")

Pretty crazy, right? Now let's close that driver. 

This should have opened up a new browser window. If you didn't see it pop up automatically, check all of your desktop displays. 

In [98]:
# Close it.
driver.close()

### 10) Use the driver to visit `www.python.org`.

### 11) Visit the OpenTable page using the driver.

Let's return to the problem at hand. We need to visit the OpenTable listings for DC. Once there, we need the HTML to load. 

In the next cell, prove you can programmatically visit the page.

In [107]:
driver.get('https://www.opentable.com/washington-dc-restaurant-listings')

### 12) Resolve the JavaScript issue using the driver and find the bookings.

What we can do in this case is:

1) Request that the page load.
2) Wait one second.
3) Grab the source HTML from the page.

The page should believe we're visiting from a live connection on a browser client, so the JavaScript should render to be part of the page source. We can then grab the page source.

**Once you have the HTML with the JavaScript rendered, repeat the processes above to find the bookings.**

In [109]:
# Import sleep:
from time import sleep

In [111]:
driver.get('https://www.opentable.com/washington-dc-restaurant-listings')



AttributeError: 'NoneType' object has no attribute 'html'

### 13) Can we get all of the items we want from the page in a single `find_all`?

To be as efficient as possible, we only want to do a single loop for each entry on the page. That means we want to find the element all of our other elements (name, location, price, and bookings) are housed within. Where is each entry located on the page?

In [ ]:
# A:

### 14) Does every entry have all of the elements we want?

In [ ]:
# A:

### 15) Use Python exceptions to handle cases when bookings aren't found.

When a booking isn't found, store `'ZERO'`.

In [ ]:
# A:

### 16) Putting it all together in a DataFrame.

**Loop through the entries. For each:**

1) Grab the relevant information we want (name, location, price, and bookings). 
2) Produce a DataFrame with the columns "name," "location," "price," and "bookings" that contains the 100 entries we'd like.

In [ ]:
# A:

### 17) [Bonus] Sending keys over the driver.

We can send keys to the page using the driver. Below is a demonstration of how to search the page using the Selenium driver.

In [52]:
# We can send keys as well. Import:
from selenium.webdriver.common.keys import Keys

In [53]:
# Open the driver.
driver = webdriver.Chrome(executable_path="../chromedriver/chromedriver")
# Visit Python.
driver.get("http://www.python.org")
# Verify we're in the right place.
assert "Python" in driver.title

In [54]:
# Find the search position.
elem = driver.find_element_by_name("q")
# Clear it.
elem.clear()
# Type in "pycon."
elem.send_keys("pycon")

In [55]:
# Send the keys.
elem.send_keys(Keys.RETURN)
# This yields no results.
assert "No results found." not in driver.page_source

In [56]:
# Close the driver.
driver.close()

In [57]:
# All at once:
driver = webdriver.Chrome(executable_path="../chromedriver/chromedriver")
driver.get("http://www.python.org")
assert "Python" in driver.title
elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source
driver.close()

## Additional Resources

---

The example above (and many others) are available in the [Selenium docs](http://selenium-python.readthedocs.io/getting-started.html).

It's especially important to explore functionality, such as [locating elements](http://selenium-python.readthedocs.io/locating-elements.html#locating-elements).

Review Selenium's [FAQs](http://selenium-python.readthedocs.io/faq.html).